# Persona Vectors - Education Scoring Experiment

**Setup:** Runtime → Change runtime type → **T4 GPU**

In [ ]:
# Clone and install
!git clone https://github.com/dxnlp/persona_vectors.git
%cd persona_vectors
!pip install -r requirements.txt -q

In [ ]:
# Verify GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0)} ({torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB)")

In [ ]:
# Set API key
import os
os.environ["OPENAI_API_KEY"] = ""  # Paste your key here

## Generate Persona Vectors

In [ ]:
# Generate positive (evil) persona responses
!python -m eval.eval_persona --model Qwen/Qwen3-4B --trait evil \
    --persona_instruction_type pos --output_path output/qwen3-4b_evil_pos.csv \
    --n_per_question 5 --judge_model gpt-4o --version extract

In [ ]:
# Generate negative (helpful) persona responses
!python -m eval.eval_persona --model Qwen/Qwen3-4B --trait evil \
    --persona_instruction_type neg --output_path output/qwen3-4b_evil_neg.csv \
    --n_per_question 5 --judge_model gpt-4o --version extract

In [ ]:
# Compute steering vectors
!python generate_vec.py --model_name Qwen/Qwen3-4B \
    --pos_path output/qwen3-4b_evil_pos.csv \
    --neg_path output/qwen3-4b_evil_neg.csv \
    --trait evil --save_dir persona_vectors/Qwen3-4B

!ls -la persona_vectors/Qwen3-4B/

## Run Education Experiment

In [ ]:
# Quick test (1 essay)
!python -m experiments.education.run_experiment --test --essays 1

In [ ]:
# Full experiment (20 essays)
!python -m experiments.education.run_experiment --essays 20

## Download Results

In [ ]:
# Download vectors and results
!zip -r results.zip persona_vectors/Qwen3-4B/ experiments/education/results/ output/

from google.colab import files
files.download('results.zip')